## Setup

In [1]:
GLOBAL_SEED = 42

import os
os.environ['PYTHONHASHSEED'] = str(GLOBAL_SEED)
import sys

import time
import gc
from tqdm import tqdm
import datetime
import pickle
import random as rnd
from glob import glob
import pandas as pd
import numpy as np
from numpy import random as np_rnd
import warnings
from math import ceil
import copy

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import rcParams
from itertools import combinations
from collections import Counter

import optuna
from optuna import Trial, create_study
from optuna.samplers import TPESampler

from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold

from sklearn.impute import KNNImputer
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from itertools import product

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from scipy.stats import f_oneway
from scipy.stats import pearsonr
from scipy.special import softmax

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import get_polynomial_decay_schedule_with_warmup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# display setting
warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False

In [2]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    # python random
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # tf random
    try:
        tf_rnd.set_seed(seed)
    except:
        pass
    # RAPIDS random
    try:
        cupy.random.seed(seed)
    except:
        pass
    # pytorch random
    try:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
    except:
        pass

def pickleIO(obj, src, op="w"):
    if op=="w":
        with open(src, op + "b") as f:
            pickle.dump(obj, f)
    elif op=="r":
        with open(src, op + "b") as f:
            tmp = pickle.load(f)
        return tmp
    else:
        print("unknown operation")
        return obj

def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)
        
def week_of_month(dt):
    """ 
        Returns the week of the month for the specified date.
    """
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + (1 + first_day.weekday()) % 7
    return int(ceil(adjusted_dom/7.0))

def quality_to_class(x, threshold):
    y_pred_class = []
    for i in x:
        tmp_class = len(threshold)
        for k, v in threshold.items():
            if i < v:
                tmp_class = k
                break
        y_pred_class.append(tmp_class)
    return y_pred_class

In [3]:
class CFG:
    debug = False
    TF = True
    product_mapper = {
        "A": ["T010305", "T010306", "T050304", "T050307"],
        "O": ["T100304", "T100306"],
        "T": ["T100304", "T100306"],
    }
    line_mapper = {
        "T010305": "A", "T010306": "A", "T050304": "A", "T050307": "A",
        "T100304": "O_T", "T100306": "O_T",
    }
    line_groups = [
        ["T010305", "T010306"],
        ["T050304", "T050307"],
        ["T100304", "T100306"],
    ]
    classes = [0, 1, 2]
    time_features = ["month", "day", "weekday", "week_of_month", "hour", "office_hour", "sec_in_day", "sin_in_day", "cos_in_day"]
    
    epochs = 20
    batch_size = 4
    early_stopping_rounds = 10
    eta = 5e-4
    weight_decay = 1e-4

In [4]:
quality_threshold = pickleIO(None, "../datasets/dataset_valid/quality_threshold.pkl", "r")
quality_threshold['T010305_T010306_T050304_T050307'] = {0: (0.525046 + 0.525086) / 2, 1: (0.534843 + 0.535279) / 2}

In [5]:
quality_threshold = {
    "T010305_T010306": {0: 0.52507, 1: 0.53490},
    "T050304_T050307": {0: 0.52507, 1: 0.53490},
    "T100304_T100306": {0: 0.52507, 1: 0.53490},
}

In [6]:
quality_threshold

{'T010305_T010306': {0: 0.52507, 1: 0.5349},
 'T050304_T050307': {0: 0.52507, 1: 0.5349},
 'T100304_T100306': {0: 0.52507, 1: 0.5349}}

## Loading Data

In [7]:
# information Provided by Dacon

# PRODUCT_ID : 제품의 고유 ID
# Y_Class : 제품 품질 상태(Target) 
# 0 : 적정 기준 미달 (부적합)
# 1 : 적합
# 2 : 적정 기준 초과 (부적합)
# Y_Quality : 제품 품질 관련 정량적 수치
# TIMESTAMP : 제품이 공정에 들어간 시각
# LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재)
# PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재)
# X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수

In [8]:
df_full = pd.read_csv("../datasets/train.csv")
df_full.columns = df_full.columns.str.lower()

In [9]:
# # time feature engineernig
# df_full["month"] = df_full["timestamp"].dt.month
# df_full["day"] = df_full["timestamp"].dt.day
# df_full["weekday"] = df_full["timestamp"].dt.weekday
# df_full["week_of_month"] = df_full["timestamp"].apply(week_of_month)
# df_full["hour"] = df_full["timestamp"].dt.hour
# df_full["office_hour"] = df_full["hour"].apply(lambda x: 1 if ((x >= 9) & (x < 18)) else 0)
# df_full["sec_in_day"] = (df_full["timestamp"] - df_full["timestamp"].dt.normalize()).dt.total_seconds() / 3600
# df_full["sin_in_day"] = np.sin(2 * np.pi * df_full["sec_in_day"].values)
# df_full["cos_in_day"] = np.cos(2 * np.pi * df_full["sec_in_day"].values)

In [10]:
# df_train = pickleIO(None, "../../datasets/dataset_valid2/df_train.pkl", "r")
# df_valid = pickleIO(None, "../../datasets/dataset_valid2/df_valid.pkl", "r")

In [11]:
# # time feature engineernig
# df_train["month"] = df_train["timestamp"].dt.month
# df_train["day"] = df_train["timestamp"].dt.day
# df_train["weekday"] = df_train["timestamp"].dt.weekday
# df_train["week_of_month"] = df_train["timestamp"].apply(week_of_month)
# df_train["hour"] = df_train["timestamp"].dt.hour
# df_train["office_hour"] = df_train["hour"].apply(lambda x: 1 if ((x >= 9) & (x < 18)) else 0)
# df_train["sec_in_day"] = (df_train["timestamp"] - df_train["timestamp"].dt.normalize()).dt.total_seconds() / 3600
# df_train["sin_in_day"] = np.sin(2 * np.pi * df_train["sec_in_day"].values)
# df_train["cos_in_day"] = np.cos(2 * np.pi * df_train["sec_in_day"].values)

In [12]:
# # time feature engineernig
# df_valid["month"] = df_valid["timestamp"].dt.month
# df_valid["day"] = df_valid["timestamp"].dt.day
# df_valid["weekday"] = df_valid["timestamp"].dt.weekday
# df_valid["week_of_month"] = df_valid["timestamp"].apply(week_of_month)
# df_valid["hour"] = df_valid["timestamp"].dt.hour
# df_valid["office_hour"] = df_valid["hour"].apply(lambda x: 1 if ((x >= 9) & (x < 18)) else 0)
# df_valid["sec_in_day"] = (df_valid["timestamp"] - df_valid["timestamp"].dt.normalize()).dt.total_seconds() / 3600
# df_valid["sin_in_day"] = np.sin(2 * np.pi * df_valid["sec_in_day"].values)
# df_valid["cos_in_day"] = np.cos(2 * np.pi * df_valid["sec_in_day"].values)

In [13]:
# df_train.info()

In [14]:
# df_train.head()

In [15]:
# df_train["y_quality"]

In [16]:
# df_train.groupby(["line", "product_code"]).size()

In [17]:
# df_valid.info()

In [18]:
# df_valid.head()

In [19]:
# df_valid["y_quality"]

In [20]:
# df_valid.groupby(["line", "product_code"]).size()

In [21]:
df_full["tmp"] = df_full["product_code"].apply(lambda x: 1 if x == "A_31" else 0)
display(df_full.groupby(["tmp", "y_class"])["y_quality"].describe().T)
df_full = df_full.drop("tmp", axis=1)

tmp              0                                 1                       
y_class          0           1          2          0           1          2
count    28.000000  289.000000  32.000000  60.000000  118.000000  71.000000
mean      0.521246    0.530272   0.538753   0.520646    0.530209   0.543508
std       0.005765    0.002334   0.004931   0.003990    0.002705   0.008733
min       0.502517    0.525213   0.534951   0.500856    0.525086   0.535279
25%       0.520467    0.528483   0.535541   0.519388    0.527989   0.537330
50%       0.523422    0.530308   0.536237   0.521315    0.530353   0.539235
75%       0.524612    0.532119   0.539517   0.523522    0.532332   0.547506
max       0.525067    0.534837   0.551279   0.525046    0.534843   0.578841

In [22]:
df_full["tmp"] = df_full["product_code"].apply(lambda x: 1 if x == "A_31" else 0)
display(df_full.groupby("tmp")["y_class"].describe().T)
df_full = df_full.drop("tmp", axis=1)

tmp,0,1
count,349.000000,249.000000
mean,1.011461,1.044177
std,0.415069,0.725442
min,0.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,1.000000,2.000000
max,2.000000,2.000000


In [23]:
df_full["tmp"] = df_full["product_code"].apply(lambda x: 1 if x == "A_31" else 0)
print(df_full.groupby("tmp")["y_class"].value_counts(normalize=True))
df_full = df_full.drop("tmp", axis=1)

tmp  y_class
0    1          0.828080
     2          0.091691
     0          0.080229
1    1          0.473896
     2          0.285141
     0          0.240964
Name: y_class, dtype: float64


In [24]:
df_test = pd.read_csv("../datasets/test.csv")
df_test.columns = df_test.columns.str.lower()

In [25]:
# # time feature engineernig
# df_test["month"] = df_test["timestamp"].dt.month
# df_test["day"] = df_test["timestamp"].dt.day
# df_test["weekday"] = df_test["timestamp"].dt.weekday
# df_test["week_of_month"] = df_test["timestamp"].apply(week_of_month)
# df_test["hour"] = df_test["timestamp"].dt.hour
# df_test["office_hour"] = df_test["hour"].apply(lambda x: 1 if ((x >= 9) & (x < 18)) else 0)
# df_test["sec_in_day"] = (df_test["timestamp"] - df_test["timestamp"].dt.normalize()).dt.total_seconds() / 3600
# df_test["sin_in_day"] = np.sin(2 * np.pi * df_test["sec_in_day"].values)
# df_test["cos_in_day"] = np.cos(2 * np.pi * df_test["sec_in_day"].values)

In [26]:
df_test.head()

,product_id,timestamp,line,product_code,x_1,x_2,x_3,x_4,x_5,x_6,...,x_2866,x_2867,x_2868,x_2869,x_2870,x_2871,x_2872,x_2873,x_2874,x_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Training by Lines

* "T010305", "T010306"
* "T050304", "T050307"
* "T100304", "T100306"

In [27]:
architecture_root_path = "./architectures/"
architecture_name = "daheeRef_bylines_dnn_jeju_try1"
architecture_path = architecture_root_path + architecture_name + "/"
createFolder(architecture_path)

In [28]:
# Shuffling
df_full = df_full.sample(frac=1, random_state=GLOBAL_SEED).reset_index(drop=True)

In [29]:
# Label Encoding on Line
lbe = LabelEncoder()
lbe.fit(["T010305", "T010306", "T050304", "T050307", "T100304", "T100306"])

LabelEncoder()

In [30]:
df_meta = {"reg_qual": None, "cls_cls": None, "reg_cls": None}

## Define helper functions

In [31]:
def get_optimizer_params(model, eta, weight_decay):
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        # apply weight decay
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'lr': eta, 'weight_decay': weight_decay},
        # don't apply weight decay for LayerNormalization layer
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
         'lr': eta, 'weight_decay': 0.0},
    ]
    return optimizer_parameters


def get_scheduler(optimizer, num_warmup_steps, num_training_steps):
    scheduler = get_polynomial_decay_schedule_with_warmup(
        optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps, power=0.5, lr_end=1e-7
    )
    return scheduler


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [32]:
# optuna function
def optuna_objective_function(trial: Trial, fold, train_x, train_y, valid_x, valid_y, output_container):
    
    tuning_params = {
        "dropoutRate": trial.suggest_float("dropoutRate", 0.2, 0.8),
        "hidden_layers": trial.suggest_int("hidden_layers", 4, 12),
        "layer_depth": trial.suggest_int("layer_depth", 2, 2),
        "nblocks": trial.suggest_int("nblocks", 2, 8),
        "batch_norm": trial.suggest_categorical("batch_norm", [True, False]),
        "activation": trial.suggest_categorical("activation", ["relu", "leakyrelu", "swish"]),
    }
    
    seed_everything(fold)
    model, score = do_fold_training(tuning_params)
    optuna_score = score["valid_loss"]
    
    if optuna_score < output_container["optuna_score"]:
        output_container["model"] = model
        output_container["optuna_score"] = optuna_score

    return optuna_score

import operator
class Optuna_EarlyStoppingCallback(object):
    """Early stopping callback for Optuna."""

    def __init__(self, early_stopping_rounds: int, direction: str = "minimize") -> None:
        self.early_stopping_rounds = early_stopping_rounds

        self._iter = 0

        if direction == "minimize":
            self._operator = operator.lt
            self._score = np.inf
        elif direction == "maximize":
            self._operator = operator.gt
            self._score = -np.inf
        else:
            ValueError(f"invalid direction: {direction}")

    def __call__(self, study: optuna.Study, trial: optuna.Trial) -> None:
        """Do early stopping."""
        if self._operator(study.best_value, self._score):
            self._iter = 0
            self._score = study.best_value
        else:
            self._iter += 1

        if self._iter >= self.early_stopping_rounds:
            study.stop()

In [33]:
class CustomDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
    def __len__(self):
        return len(self.feature)        
    def __getitem__(self, idx):
        return {'feature': self.feature[idx], 'label': self.label[idx]}

class LinearBlock(nn.Module):
    def __init__(self, input_shape, dropoutRate=1/2, hidden_layers=16, layer_depth=2):
        super(LinearBlock, self).__init__()
        self.model = nn.Sequential(
            nn.Dropout(dropoutRate),
            *[nn.Sequential(
                nn.Linear((input_shape if i == 0 else hidden_layers), hidden_layers),
                nn.ReLU()
            ) for i in range(layer_depth)],
        )
    def forward(self, features):
        return self.model(features)

class ResidualLinearBlock(nn.Module):
    def __init__(self, input_shape, dropoutRate=1/2, hidden_layers=16, layer_depth=2, batch_norm=False):
        super(ResidualLinearBlock, self).__init__()
        
        if batch_norm:
            self.pool = nn.Sequential(
                nn.Linear(input_shape, hidden_layers),
                nn.BatchNorm1d(hidden_layers),
                nn.ReLU(),
            )

            self.resd = nn.Sequential(
                nn.Dropout(dropoutRate),
                nn.Linear(hidden_layers, hidden_layers),
                nn.BatchNorm1d(hidden_layers),
                nn.ReLU(),
                nn.Linear(hidden_layers, hidden_layers),
                nn.BatchNorm1d(hidden_layers),
            )
        else:
            self.pool = nn.Sequential(
                nn.Linear(input_shape, hidden_layers),
                nn.ReLU(),
            )

            self.resd = nn.Sequential(
                nn.Dropout(dropoutRate),
                nn.Linear(hidden_layers, hidden_layers),
                nn.ReLU(),
                nn.Linear(hidden_layers, hidden_layers),
            )           
        
    def forward(self, features):
        x = self.pool(features)
        return F.relu(self.resd(x) + x)
    
class GatedLinearBlock(nn.Module):
    def __init__(self, input_shape, dropoutRate=1/2, hidden_layers=16, layer_depth=2, batch_norm=False, activation="relu"):
        super(GatedLinearBlock, self).__init__()
        if activation == "leakyrelu":
            act = nn.LeakyReLU()
        elif activation == "swish":
            act = nn.SiLU()
        else:
            act = nn.ReLU()
        
        if batch_norm:
            self.pool = nn.Sequential(
                nn.Linear(input_shape, hidden_layers),
                nn.BatchNorm1d(hidden_layers),
            )

            self.gated_lin = nn.Sequential(
                nn.Dropout(dropoutRate),
                nn.Linear(hidden_layers, hidden_layers),
                nn.Sigmoid(),
            )
            
            self.lin = nn.Sequential(
                nn.Dropout(dropoutRate),
                nn.Linear(hidden_layers * 2, hidden_layers),
                nn.BatchNorm1d(hidden_layers),
                act,
                nn.Linear(hidden_layers, hidden_layers),
                nn.BatchNorm1d(hidden_layers),
                act,
            )
        else:
            self.pool = nn.Sequential(
                nn.Linear(input_shape, hidden_layers)
            )

            self.gated_lin = nn.Sequential(
                nn.Dropout(dropoutRate),
                nn.Linear(hidden_layers, hidden_layers),
                nn.Sigmoid(),
            )
            
            self.lin = nn.Sequential(
                nn.Dropout(dropoutRate),
                nn.Linear(hidden_layers * 2, hidden_layers),
                act,
                nn.Linear(hidden_layers, hidden_layers),
                act,
            )
            
    def forward(self, features):
        x_pool = self.pool(features)
        x_gated_lin = x_pool * self.gated_lin(x_pool)
        x_concat = torch.cat([x_pool, x_gated_lin], dim=-1)
        # Is skip connection is needed?
        return self.lin(x_concat) + x_pool

### Training with tuning parameters - Regression (y_quality) 

In [34]:
def train_fn(model, dl, criterion, optimizer, scheduler, grad_scaler):
    model.train()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.4f"),
    }
    
    for idx, batch in enumerate(dl):
        output_topic = model(batch["feature"].to(torch.int64))
        loss = criterion(output_topic.flatten(), batch["label"])
        # initialization gradients to zero
        optimizer.zero_grad()
        # backprop
        loss.backward()
        # update learning rate
        optimizer.step()
        metrics["loss"].update(loss.item())
        
        if CFG.debug:
            if idx >= 4: break
    
    # update scheduler
    scheduler.step()

    return metrics

def valid_fn(model, dl, criterion):
    model.eval()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.6f"),
    }   
    for idx, batch in enumerate(dl):
        with torch.no_grad():
            output_topic = model(batch["feature"].to(torch.int64))
            loss = criterion(output_topic.flatten(), batch["label"])
        # update metric value
        metrics["loss"].update(loss.item())
    
        if CFG.debug:
            if idx >= 4: break
            
    return metrics

def do_fold_training(BEST_PARAMS={}):
        
    if len(BEST_PARAMS) == 0:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        BEST_PARAMS["dropoutRate"] = 0.5
        BEST_PARAMS["hidden_layers"] = 32
        BEST_PARAMS["layer_depth"] = 2
        BEST_PARAMS["nblocks"] = 2
    else:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        
    model = CustomModel(BEST_PARAMS)
    train_dl = DataLoader(CustomDataset(torch.from_numpy(train_x).to(torch.float32), torch.from_numpy(train_y).to(torch.float32)), shuffle=True, batch_size=CFG.batch_size, drop_last=True)
    valid_dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.from_numpy(valid_y).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)

    # set optimizer
    optimizer_parameters = get_optimizer_params(
        model,
        eta=CFG.eta,
        weight_decay=CFG.weight_decay
    )
    optimizer = AdamW(optimizer_parameters, lr=CFG.eta, weight_decay=CFG.weight_decay)
    scheduler = get_scheduler(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=CFG.epochs
    )
#     scheduler = None
    grad_scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    criterion = nn.MSELoss(reduction="mean")
    best_score = np.inf

    early_stopping_cnt = 0
    for epoch in range(CFG.epochs):
        epoch_start_time = time.time()
        # training
        train_metrics = train_fn(model, train_dl, criterion, optimizer, scheduler, grad_scaler)
        # evaluation
        valid_metrics = valid_fn(model, valid_dl, criterion)

        score = valid_metrics["loss"].avg
#         print("Epoch[{0}/{1}]\n train loss : {2}\n valid loss : {3}\n train accuracy : {4}\n valid accuracy : {5}\n eta : {6}\n Elapsed : {7}\n"
#               .format(
#                   epoch+1, CFG.epochs,
#                   round(train_metrics["loss"].avg, 5), round(valid_metrics["loss"].avg, 5),
#                   round(train_metrics["accuracy"].avg, 5), round(valid_metrics["accuracy"].avg, 5),
#                   round(optimizer.param_groups[-1]['lr'], 5), round(time.time() - epoch_start_time, 3)
#               )
#         )
#         print("Epoch[{0}/{1}]\n train loss : {2}\n valid loss : {3}\n eta : {4}\n Elapsed : {5}\n"
#               .format(
#                   epoch+1, CFG.epochs,
#                   round(train_metrics["loss"].avg, 6), round(valid_metrics["loss"].avg, 6),
#                   round(optimizer.param_groups[-1]['lr'], 5), round(time.time() - epoch_start_time, 3)
#               )
#         )
    
        if score < best_score:
            best_score = score
            return_score_dic = {
                "fold": fold,
                "train_loss": train_metrics["loss"].avg,
                "valid_loss": valid_metrics["loss"].avg,
#                 "train_accuracy": train_metrics["accuracy"].avg,
#                 "valid_accuracy": valid_metrics["accuracy"].avg,
            }
            model_save_dic = {'model': model.state_dict()}
            early_stopping_cnt = 0
        else:
            early_stopping_cnt += 1
        
        if early_stopping_cnt == CFG.early_stopping_rounds:
#             print("INFO : Early Stopped ! (Epoch[{0}/{1}])".format(epoch+1, CFG.epochs))
            break
        
    torch.save(
        model_save_dic,
        f"./model_fold{fold}_best.pth",
    )
    model.load_state_dict(torch.load(f"./model_fold{fold}_best.pth")["model"])
    model.eval()

    return copy.deepcopy(model), return_score_dic

In [35]:
class CustomModel(nn.Module):
    def __init__(self, params):
        super(CustomModel, self).__init__()
        self.params = params
        
        self.embedding = nn.Sequential(
            nn.Embedding(params["nfeatures"], embed_dim),
            nn.Flatten(),
        )
        
        self.model = nn.Sequential(
            *[GatedLinearBlock(
                (params["nfeatures"] * embed_dim) if i == 0 else max(4, int(params["hidden_layers"] / (2**(i-1)))),
                dropoutRate=params["dropoutRate"],
                hidden_layers=max(4, int(params["hidden_layers"] / (2**i))),
                layer_depth=params["layer_depth"],
                batch_norm=params["batch_norm"],
                activation=params["activation"]) for i in range(params["nblocks"])
             ],
        )

        self.regressor = nn.Linear(list(self.model.named_parameters())[-1][1].shape[0], 1)
        
    def forward(self, feature):
        x = self.embedding(feature)
        x = self.model(x)
        return self.regressor(x)

In [36]:
# fake_params = {}
# fake_params["nfeatures"] = full_x.shape[-1]
# fake_params["dropoutRate"] = 0.5
# fake_params["hidden_layers"] = 32
# fake_params["layer_depth"] = 2
# fake_params["nblocks"] = 2
# fake_params["batch_norm"] = False

# tmp = CustomModel(fake_params)
# tmp

In [37]:
kbins_dim = 128
embed_dim = 32
line_split = [
    # A
    ["A_31"],
    # O, T
    ["T_31", "O_31"],
]
score_dic = {
    "mae": None,
    "r2": None,
    "accuracy": None,
    "f1": None,
}
line_split_f1 = {}
line_score = {"_".join(i): {"train": score_dic.copy(), "valid": score_dic.copy()} for i in line_split}
valid_pred = {"_".join(i): None for i in line_split}
test_pred = {"_".join(i): None for i in line_split}

for line in line_split:
    seed_everything()
    
    # Training
    # === Preprocessing (Train) ===
    full_x = df_full[df_full["product_code"].isin(line)]
    full_x["line"] = lbe.transform(full_x["line"])
    
    full_y = full_x["y_quality"].values
    full_y_cls = full_x["y_class"].values
    # Drop columns
    full_x = full_x.drop(["product_id", "y_class", "y_quality", "timestamp", "product_code"], axis=1)
    
    full_x = full_x[full_x.columns[~(full_x.var() == 0).values & ~full_x.isna().all().values]]
    full_x = full_x.fillna(-1.0)
    full_x = full_x.T.drop_duplicates().T
    full_x = full_x.drop_duplicates().reset_index(drop=True)
    
    selected_vars = full_x.columns
    cat_vars = ["line"]
    num_vars = diff(selected_vars, cat_vars)
    categoIdx = findIdx(selected_vars, cat_vars)

    full_x[num_vars] = full_x[num_vars].astype("float32")
    full_x[cat_vars] = full_x[cat_vars].astype("int32")
    
    full_x_embed = full_x.copy()
    kbins = KBinsDiscretizer(kbins_dim, encode="ordinal", strategy="uniform")
    full_x_embed[num_vars] = kbins.fit_transform(full_x[num_vars])
    full_x_embed = full_x_embed.astype("int64").to_numpy()

    scaler = MinMaxScaler()
    full_x = scaler.fit_transform(full_x)
    print(full_x.shape)
    
    # === Preprocessing (Test) ===
    test_x = df_test[df_test["product_code"].isin(line)]
    test_x["line"] = lbe.transform(test_x["line"])
    test_x = test_x[selected_vars]
    test_x = test_x.fillna(-1.0)

    test_x_embed = test_x.copy()
    test_x_embed[num_vars] = kbins.transform(test_x[num_vars])
    test_x_embed = test_x_embed.astype("int64").to_numpy()
    
    test_x[num_vars] = test_x[num_vars].astype("float32")
    test_x[cat_vars] = test_x[cat_vars].astype("int32")
    test_x = scaler.transform(test_x)
#     break
    
    # === Neural Architecture Search with Optuna & Training ===
    line_full_pred = np.zeros(len(full_x))
    line_test_pred = np.zeros(len(test_x))
    kfolds_spliter = StratifiedKFold(3 if CFG.debug else 5, shuffle=True, random_state=42)
    for fold, (train_idx, valid_idx) in enumerate(kfolds_spliter.split(full_x, full_y_cls)):

        train_x = full_x_embed[train_idx]
        train_y = np.log1p(full_y[train_idx])
        valid_x = full_x_embed[valid_idx]
        valid_y = np.log1p(full_y[valid_idx])
#         break
        output_container = {"model": None, "optuna_score": np.inf}
        optuna_direction = "minimize"
        optuna_trials = 3 if CFG.debug else 100
        optuna_earlyStopping = Optuna_EarlyStoppingCallback(max(5, 100), direction=optuna_direction)
        optuna_timout = 6 * 3600
        optuna_study = create_study(direction=optuna_direction, sampler=TPESampler(seed=GLOBAL_SEED))

        optuna_study.optimize(
            lambda trial: optuna_objective_function(
                trial, GLOBAL_SEED,
                train_x, train_y, valid_x, valid_y, output_container=output_container,
            ),
            n_jobs=1, n_trials=optuna_trials, timeout=optuna_timout, callbacks=[optuna_earlyStopping]
        )

        model = output_container["model"]
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.zeros(len(valid_x)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"].to(torch.int64)).detach().cpu().numpy().flatten())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_full_pred[valid_idx] = np.expm1(tmp_pred)
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(test_x_embed).to(torch.float32), torch.zeros(len(test_x_embed)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"].to(torch.int64)).detach().cpu().numpy().flatten())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_test_pred[:] += np.expm1(tmp_pred) / kfolds_spliter.n_splits

    # Evaluation
    y_true = full_y.copy()
    y_pred = line_full_pred.copy()
    eval_mae = mean_absolute_error(y_true, y_pred)
    eval_r2 = r2_score(y_true, y_pred)

    # Transform quality to class
    y_true_class = full_y_cls.copy()
    y_pred_class = quality_to_class(y_pred, {0: 0.52507, 1: 0.53490})
    eval_acc = accuracy_score(y_true_class, y_pred_class)
    eval_f1 = f1_score(y_true_class, y_pred_class, average="macro")
    
    # Save values
    line_score["_".join(line)]["valid"]["mae"] = eval_mae
    line_score["_".join(line)]["valid"]["r2"] = eval_r2
    line_score["_".join(line)]["valid"]["accuracy"] = eval_acc
    line_score["_".join(line)]["valid"]["f1"] = eval_f1
    valid_pred["_".join(line)] = y_pred
    
    # Inference
    y_pred = line_test_pred.copy()
    
    # Save values
    test_pred["_".join(line)] = y_pred

df_meta["reg_qual"] = [valid_pred, test_pred]

for tmp_model in glob("./model_fold*"):
    os.remove(tmp_model)

(249, 1173)


[I 2023-02-17 00:50:58,233] A new study created in memory with name: no-name-e4f5217a-3b5f-4268-ad84-3ecfd0093664
[I 2023-02-17 00:51:16,759] Trial 0 finished with value: 7.245400967384249e-05 and parameters: {'dropoutRate': 0.42472407130841755, 'hidden_layers': 12, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True, 'activation': 'swish'}. Best is trial 0 with value: 7.245400967384249e-05.
[I 2023-02-17 00:51:25,301] Trial 1 finished with value: 0.020500278530212548 and parameters: {'dropoutRate': 0.5606690070459254, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': True, 'activation': 'relu'}. Best is trial 0 with value: 7.245400967384249e-05.
[I 2023-02-17 00:51:34,665] Trial 2 finished with value: 0.0003056847207727305 and parameters: {'dropoutRate': 0.38254534577572263, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'swish'}. Best is trial 0 with value: 7.245400967384249e-05.
[I 2023-02-17 00:51:42,961] Trial 3 finished with

[I 2023-02-17 00:56:00,780] Trial 28 finished with value: 9.347778527276554e-05 and parameters: {'dropoutRate': 0.7365395002411009, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 15 with value: 3.440275358675885e-05.
[I 2023-02-17 00:56:10,541] Trial 29 finished with value: 0.0001231739233144953 and parameters: {'dropoutRate': 0.610306322546282, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 15 with value: 3.440275358675885e-05.
[I 2023-02-17 00:56:21,499] Trial 30 finished with value: 2.7808043927717125e-05 and parameters: {'dropoutRate': 0.5050076681503559, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'swish'}. Best is trial 30 with value: 2.7808043927717125e-05.
[I 2023-02-17 00:56:32,283] Trial 31 finished with value: 2.8708782268110484e-05 and parameters: {'dropoutRate': 0.4459069328718809, 'hidden_layers': 6, 

[I 2023-02-17 01:01:14,626] Trial 56 finished with value: 0.003165982484530944 and parameters: {'dropoutRate': 0.4885422729305921, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True, 'activation': 'swish'}. Best is trial 32 with value: 2.755595529344226e-05.
[I 2023-02-17 01:01:25,617] Trial 57 finished with value: 4.3360848534435856e-05 and parameters: {'dropoutRate': 0.5446073200664514, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'swish'}. Best is trial 32 with value: 2.755595529344226e-05.
[I 2023-02-17 01:01:37,407] Trial 58 finished with value: 4.299708619398119e-05 and parameters: {'dropoutRate': 0.517243851688992, 'hidden_layers': 11, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 32 with value: 2.755595529344226e-05.
[I 2023-02-17 01:01:48,213] Trial 59 finished with value: 2.8129956000646835e-05 and parameters: {'dropoutRate': 0.3795866491921135, 'hidden_layers': 6, 'layer

[I 2023-02-17 01:06:18,470] Trial 84 finished with value: 2.708282138778136e-05 and parameters: {'dropoutRate': 0.2728215209548886, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'swish'}. Best is trial 76 with value: 2.6264707545264035e-05.
[I 2023-02-17 01:06:29,220] Trial 85 finished with value: 0.0003342803712047708 and parameters: {'dropoutRate': 0.24153942971131942, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'swish'}. Best is trial 76 with value: 2.6264707545264035e-05.
[I 2023-02-17 01:06:35,734] Trial 86 finished with value: 4.516678981102674e-05 and parameters: {'dropoutRate': 0.2828415843981079, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 76 with value: 2.6264707545264035e-05.
[I 2023-02-17 01:06:46,560] Trial 87 finished with value: 0.00013503495546711536 and parameters: {'dropoutRate': 0.24548277281126213, 'hidden_layers'

...

[I 2023-02-17 03:37:23,382] Trial 64 finished with value: 9.140510573817866e-06 and parameters: {'dropoutRate': 0.6282212882961941, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 57 with value: 8.92532946434383e-06.
[I 2023-02-17 03:37:34,447] Trial 65 finished with value: 0.0004995423283212909 and parameters: {'dropoutRate': 0.6192642926928121, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'relu'}. Best is trial 57 with value: 8.92532946434383e-06.
[I 2023-02-17 03:37:45,856] Trial 66 finished with value: 9.187991148943789e-06 and parameters: {'dropoutRate': 0.640607145563578, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 57 with value: 8.92532946434383e-06.
[I 2023-02-17 03:37:59,909] Trial 67 finished with value: 0.0003967784314227174 and parameters: {'dropoutRate': 0.6419633730994572, 'hidden_layers': 7, 'layer_depth

[I 2023-02-17 03:43:11,147] Trial 93 finished with value: 9.019921220796176e-06 and parameters: {'dropoutRate': 0.5954187995007634, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 83 with value: 8.822983684492809e-06.
[I 2023-02-17 03:43:23,877] Trial 94 finished with value: 9.091189636162906e-06 and parameters: {'dropoutRate': 0.5409375667956142, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 83 with value: 8.822983684492809e-06.
[I 2023-02-17 03:43:36,419] Trial 95 finished with value: 9.487803917333318e-06 and parameters: {'dropoutRate': 0.5121380751302288, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 83 with value: 8.822983684492809e-06.
[I 2023-02-17 03:43:48,875] Trial 96 finished with value: 1.4660459029869344e-05 and parameters: {'dropoutRate': 0.5475774100066437, 'hidden_layers': 9, 'layer_

[I 2023-02-17 03:48:34,749] Trial 21 finished with value: 1.3009251820979747e-05 and parameters: {'dropoutRate': 0.20246274045404689, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'swish'}. Best is trial 21 with value: 1.3009251820979747e-05.
[I 2023-02-17 03:48:47,491] Trial 22 finished with value: 1.2899977933658294e-05 and parameters: {'dropoutRate': 0.20066981916795848, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'swish'}. Best is trial 22 with value: 1.2899977933658294e-05.
[I 2023-02-17 03:49:00,106] Trial 23 finished with value: 1.2672987818657728e-05 and parameters: {'dropoutRate': 0.3079323256068981, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'swish'}. Best is trial 23 with value: 1.2672987818657728e-05.
[I 2023-02-17 03:49:12,790] Trial 24 finished with value: 1.2153670708256766e-05 and parameters: {'dropoutRate': 0.28831901561493256, 'hidden_layers'

[I 2023-02-17 03:54:46,685] Trial 49 finished with value: 0.00032910103207781403 and parameters: {'dropoutRate': 0.49916531695638056, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'swish'}. Best is trial 47 with value: 9.286092729925915e-06.
[I 2023-02-17 03:54:57,636] Trial 50 finished with value: 9.126017125203765e-06 and parameters: {'dropoutRate': 0.4236566650375127, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 50 with value: 9.126017125203765e-06.
[I 2023-02-17 03:55:08,448] Trial 51 finished with value: 9.319635686387705e-06 and parameters: {'dropoutRate': 0.412376520722588, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 50 with value: 9.126017125203765e-06.
[I 2023-02-17 03:55:19,901] Trial 52 finished with value: 9.333061244534394e-06 and parameters: {'dropoutRate': 0.41153021322229466, 'hidden_layers'

[I 2023-02-17 04:00:41,831] Trial 77 finished with value: 0.014410986772014035 and parameters: {'dropoutRate': 0.4738569494129032, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True, 'activation': 'leakyrelu'}. Best is trial 62 with value: 8.884220594356722e-06.
[I 2023-02-17 04:00:53,156] Trial 78 finished with value: 0.00012483001715205924 and parameters: {'dropoutRate': 0.36651571913892506, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'relu'}. Best is trial 62 with value: 8.884220594356722e-06.
[I 2023-02-17 04:01:05,693] Trial 79 finished with value: 0.00048093710938701406 and parameters: {'dropoutRate': 0.49835267762303637, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 62 with value: 8.884220594356722e-06.
[I 2023-02-17 04:01:16,867] Trial 80 finished with value: 3.559738908936399e-05 and parameters: {'dropoutRate': 0.34101551041070416, 'hidden_layers'

In [38]:
print(line_score)

{'A_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.005508148792026257, 'r2': 0.3462220535108984, 'accuracy': 0.6184738955823293, 'f1': 0.5712418300653596}}, 'T_31_O_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.0036440269941266333, 'r2': -0.11774531845618252, 'accuracy': 0.7736389684813754, 'f1': 0.4165266106442577}}}


### Training - Regression (y_class)

In [39]:
def train_fn(model, dl, criterion, optimizer, scheduler, grad_scaler):
    model.train()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.4f"),
    }
    
    for idx, batch in enumerate(dl):
        output_topic = model(batch["feature"].to(torch.int64))
        loss = criterion(output_topic.flatten(), batch["label"])
        # initialization gradients to zero
        optimizer.zero_grad()
        # backprop
        loss.backward()
        # update learning rate
        optimizer.step()
        metrics["loss"].update(loss.item())
        
        if CFG.debug:
            if idx >= 4: break
    
    # update scheduler
    scheduler.step()

    return metrics

def valid_fn(model, dl, criterion):
    model.eval()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.6f"),
    }   
    for idx, batch in enumerate(dl):
        with torch.no_grad():
            output_topic = model(batch["feature"].to(torch.int64))
            loss = criterion(output_topic.flatten(), batch["label"])
        # update metric value
        metrics["loss"].update(loss.item())
    
        if CFG.debug:
            if idx >= 4: break
            
    return metrics

def do_fold_training(BEST_PARAMS={}):
        
    if len(BEST_PARAMS) == 0:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        BEST_PARAMS["dropoutRate"] = 0.5
        BEST_PARAMS["hidden_layers"] = 32
        BEST_PARAMS["layer_depth"] = 2
        BEST_PARAMS["nblocks"] = 2
    else:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        
    model = CustomModel(BEST_PARAMS)
    train_dl = DataLoader(CustomDataset(torch.from_numpy(train_x).to(torch.float32), torch.from_numpy(train_y).to(torch.float32)), shuffle=True, batch_size=CFG.batch_size, drop_last=True)
    valid_dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.from_numpy(valid_y).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)

    # set optimizer
    optimizer_parameters = get_optimizer_params(
        model,
        eta=CFG.eta,
        weight_decay=CFG.weight_decay
    )
    optimizer = AdamW(optimizer_parameters, lr=CFG.eta, weight_decay=CFG.weight_decay)
    scheduler = get_scheduler(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=CFG.epochs
    )
#     scheduler = None
    grad_scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    criterion = nn.MSELoss(reduction="mean")
    best_score = np.inf

    early_stopping_cnt = 0
    for epoch in range(CFG.epochs):
        epoch_start_time = time.time()
        # training
        train_metrics = train_fn(model, train_dl, criterion, optimizer, scheduler, grad_scaler)
        # evaluation
        valid_metrics = valid_fn(model, valid_dl, criterion)

        score = valid_metrics["loss"].avg
#         print("Epoch[{0}/{1}]\n train loss : {2}\n valid loss : {3}\n train accuracy : {4}\n valid accuracy : {5}\n eta : {6}\n Elapsed : {7}\n"
#               .format(
#                   epoch+1, CFG.epochs,
#                   round(train_metrics["loss"].avg, 5), round(valid_metrics["loss"].avg, 5),
#                   round(train_metrics["accuracy"].avg, 5), round(valid_metrics["accuracy"].avg, 5),
#                   round(optimizer.param_groups[-1]['lr'], 5), round(time.time() - epoch_start_time, 3)
#               )
#         )
#         print("Epoch[{0}/{1}]\n train loss : {2}\n valid loss : {3}\n eta : {4}\n Elapsed : {5}\n"
#               .format(
#                   epoch+1, CFG.epochs,
#                   round(train_metrics["loss"].avg, 6), round(valid_metrics["loss"].avg, 6),
#                   round(optimizer.param_groups[-1]['lr'], 5), round(time.time() - epoch_start_time, 3)
#               )
#         )
    
        if score < best_score:
            best_score = score
            return_score_dic = {
                "fold": fold,
                "train_loss": train_metrics["loss"].avg,
                "valid_loss": valid_metrics["loss"].avg,
#                 "train_accuracy": train_metrics["accuracy"].avg,
#                 "valid_accuracy": valid_metrics["accuracy"].avg,
            }
            model_save_dic = {'model': model.state_dict()}
            early_stopping_cnt = 0
        else:
            early_stopping_cnt += 1
        
        if early_stopping_cnt == CFG.early_stopping_rounds:
#             print("INFO : Early Stopped ! (Epoch[{0}/{1}])".format(epoch+1, CFG.epochs))
            break
        
    torch.save(
        model_save_dic,
        f"./model_fold{fold}_best.pth",
    )
    model.load_state_dict(torch.load(f"./model_fold{fold}_best.pth")["model"])
    model.eval()

    return copy.deepcopy(model), return_score_dic

In [40]:
class CustomModel(nn.Module):
    def __init__(self, params):
        super(CustomModel, self).__init__()
        self.params = params
        
        self.embedding = nn.Sequential(
            nn.Embedding(params["nfeatures"], embed_dim),
            nn.Flatten(),
        )
        
        self.model = nn.Sequential(
            *[GatedLinearBlock(
                (params["nfeatures"] * embed_dim) if i == 0 else max(4, int(params["hidden_layers"] / (2**(i-1)))),
                dropoutRate=params["dropoutRate"],
                hidden_layers=max(4, int(params["hidden_layers"] / (2**i))),
                layer_depth=params["layer_depth"],
                batch_norm=params["batch_norm"],
                activation=params["activation"]) for i in range(params["nblocks"])
             ],
        )

        self.regressor = nn.Linear(list(self.model.named_parameters())[-1][1].shape[0], 1)
        
    def forward(self, feature):
        x = self.embedding(feature)
        x = self.model(x)
        return self.regressor(x)

In [41]:
kbins_dim = 128
embed_dim = 32
line_split = [
    # A
    ["A_31"],
    # O, T
    ["T_31", "O_31"],
]
score_dic = {
    "mae": None,
    "r2": None,
    "accuracy": None,
    "f1": None,
}
line_split_f1 = {}
line_score = {"_".join(i): {"train": score_dic.copy(), "valid": score_dic.copy()} for i in line_split}
valid_pred = {"_".join(i): None for i in line_split}
test_pred = {"_".join(i): None for i in line_split}

for line in line_split:
    seed_everything()
    
    # Training
    # === Preprocessing (Train) ===
    full_x = df_full[df_full["product_code"].isin(line)]
    full_x["line"] = lbe.transform(full_x["line"])
    
    full_y = full_x["y_class"].values
    full_y_cls = full_x["y_class"].values
    # Drop columns
    full_x = full_x.drop(["product_id", "y_class", "y_quality", "timestamp", "product_code"], axis=1)
    
    full_x = full_x[full_x.columns[~(full_x.var() == 0).values & ~full_x.isna().all().values]]
    full_x = full_x.fillna(-1.0)
    full_x = full_x.T.drop_duplicates().T
    full_x = full_x.drop_duplicates().reset_index(drop=True)
    
    selected_vars = full_x.columns
    cat_vars = ["line"]
    num_vars = diff(selected_vars, cat_vars)
    categoIdx = findIdx(selected_vars, cat_vars)

    full_x[num_vars] = full_x[num_vars].astype("float32")
    full_x[cat_vars] = full_x[cat_vars].astype("int32")
    
    full_x_embed = full_x.copy()
    kbins = KBinsDiscretizer(kbins_dim, encode="ordinal", strategy="uniform")
    full_x_embed[num_vars] = kbins.fit_transform(full_x[num_vars])
    full_x_embed = full_x_embed.astype("int64").to_numpy()

    scaler = MinMaxScaler()
    full_x = scaler.fit_transform(full_x)
    print(full_x.shape)
    
    # === Preprocessing (Test) ===
    test_x = df_test[df_test["product_code"].isin(line)]
    test_x["line"] = lbe.transform(test_x["line"])
    test_x = test_x[selected_vars]
    test_x = test_x.fillna(-1.0)

    test_x_embed = test_x.copy()
    test_x_embed[num_vars] = kbins.transform(test_x[num_vars])
    test_x_embed = test_x_embed.astype("int64").to_numpy()
    
    test_x[num_vars] = test_x[num_vars].astype("float32")
    test_x[cat_vars] = test_x[cat_vars].astype("int32")
    test_x = scaler.transform(test_x)
#     break
    
    # === Neural Architecture Search with Optuna & Training ===
    line_full_pred = np.zeros(len(full_x))
    line_test_pred = np.zeros(len(test_x))
    kfolds_spliter = StratifiedKFold(3 if CFG.debug else 5, shuffle=True, random_state=42)
    for fold, (train_idx, valid_idx) in enumerate(kfolds_spliter.split(full_x, full_y_cls)):

        train_x = full_x_embed[train_idx]
        train_y = full_y[train_idx]
        valid_x = full_x_embed[valid_idx]
        valid_y = full_y[valid_idx]
#         break
        output_container = {"model": None, "optuna_score": np.inf}
        optuna_direction = "minimize"
        optuna_trials = 3 if CFG.debug else 100
        optuna_earlyStopping = Optuna_EarlyStoppingCallback(max(5, 100), direction=optuna_direction)
        optuna_timout = 6 * 3600
        optuna_study = create_study(direction=optuna_direction, sampler=TPESampler(seed=GLOBAL_SEED))

        optuna_study.optimize(
            lambda trial: optuna_objective_function(
                trial, GLOBAL_SEED,
                train_x, train_y, valid_x, valid_y, output_container=output_container,
            ),
            n_jobs=1, n_trials=optuna_trials, timeout=optuna_timout, callbacks=[optuna_earlyStopping]
        )

        model = output_container["model"]
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.zeros(len(valid_x)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"].to(torch.int64)).detach().cpu().numpy().flatten())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_full_pred[valid_idx] = tmp_pred
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(test_x_embed).to(torch.float32), torch.zeros(len(test_x_embed)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"].to(torch.int64)).detach().cpu().numpy().flatten())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_test_pred[:] += tmp_pred / kfolds_spliter.n_splits

    # Evaluation
    y_true = full_y.copy()
    y_pred = line_full_pred.copy()
    eval_mae = mean_absolute_error(y_true, y_pred)
    eval_r2 = r2_score(y_true, y_pred)

    # Transform quality to class
    y_true_class = y_true.copy()
    y_pred_class = quality_to_class(y_pred, {0: 0.5, 1: 1.5})
    eval_acc = accuracy_score(y_true_class, y_pred_class)
    eval_f1 = f1_score(y_true_class, y_pred_class, average="macro")
    
    # Save values
    line_score["_".join(line)]["valid"]["mae"] = eval_mae
    line_score["_".join(line)]["valid"]["r2"] = eval_r2
    line_score["_".join(line)]["valid"]["accuracy"] = eval_acc
    line_score["_".join(line)]["valid"]["f1"] = eval_f1
    valid_pred["_".join(line)] = y_pred
    
    # Inference
    y_pred = line_test_pred.copy()
    
    # Save values
    test_pred["_".join(line)] = y_pred

df_meta["reg_cls"] = [valid_pred, test_pred]

for tmp_model in glob("./model_fold*"):
    os.remove(tmp_model)

(249, 1173)


[I 2023-02-17 05:24:59,370] A new study created in memory with name: no-name-13865ef1-370c-456b-bc1a-a9e641ae1390
[I 2023-02-17 05:25:18,274] Trial 0 finished with value: 0.6348289171090493 and parameters: {'dropoutRate': 0.42472407130841755, 'hidden_layers': 12, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True, 'activation': 'swish'}. Best is trial 0 with value: 0.6348289171090493.
[I 2023-02-17 05:25:26,425] Trial 1 finished with value: 0.8434204848913046 and parameters: {'dropoutRate': 0.5606690070459254, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': True, 'activation': 'relu'}. Best is trial 0 with value: 0.6348289171090493.
[I 2023-02-17 05:25:34,190] Trial 2 finished with value: 0.2540070219681813 and parameters: {'dropoutRate': 0.38254534577572263, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'swish'}. Best is trial 2 with value: 0.2540070219681813.
[I 2023-02-17 05:25:42,264] Trial 3 finished with value: 0.2797420

[I 2023-02-17 05:30:36,494] Trial 29 finished with value: 0.6523914188146591 and parameters: {'dropoutRate': 0.4928002038618747, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True, 'activation': 'relu'}. Best is trial 23 with value: 0.24807737452479509.
[I 2023-02-17 05:30:48,061] Trial 30 finished with value: 0.26740611688448834 and parameters: {'dropoutRate': 0.6391501656697286, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 23 with value: 0.24807737452479509.
[I 2023-02-17 05:30:59,051] Trial 31 finished with value: 0.25895481699934375 and parameters: {'dropoutRate': 0.404557448428499, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'relu'}. Best is trial 23 with value: 0.24807737452479509.
[I 2023-02-17 05:31:09,581] Trial 32 finished with value: 0.2516070114305386 and parameters: {'dropoutRate': 0.4395675335181002, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks

[I 2023-02-17 05:36:03,886] Trial 58 finished with value: 0.25981911844932115 and parameters: {'dropoutRate': 0.44809158973760144, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 43 with value: 0.24728955729649618.
[I 2023-02-17 05:36:21,326] Trial 59 finished with value: 0.7141663466508572 and parameters: {'dropoutRate': 0.5118042621728519, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True, 'activation': 'leakyrelu'}. Best is trial 43 with value: 0.24728955729649618.
[I 2023-02-17 05:36:27,185] Trial 60 finished with value: 0.2579919627079597 and parameters: {'dropoutRate': 0.2875118937141797, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 3, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 43 with value: 0.24728955729649618.
[I 2023-02-17 05:36:38,100] Trial 61 finished with value: 0.24811367977123994 and parameters: {'dropoutRate': 0.3881809017842597, 'hidden_layers': 4, 'layer_de

[I 2023-02-17 05:41:01,191] Trial 86 finished with value: 0.253133874386549 and parameters: {'dropoutRate': 0.36752152647996106, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 74 with value: 0.2466097531410364.
[I 2023-02-17 05:41:11,660] Trial 87 finished with value: 0.25097217009617734 and parameters: {'dropoutRate': 0.31680136453860214, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 74 with value: 0.2466097531410364.
[I 2023-02-17 05:41:26,526] Trial 88 finished with value: 0.8047376767947123 and parameters: {'dropoutRate': 0.35001978008414625, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True, 'activation': 'leakyrelu'}. Best is trial 74 with value: 0.2466097531410364.
[I 2023-02-17 05:41:37,082] Trial 89 finished with value: 0.2485655422967214 and parameters: {'dropoutRate': 0.3934544406763592, 'hidden_layers': 4, 'layer_depth

...

[I 2023-02-17 07:37:25,618] Trial 70 finished with value: 0.16099654738273886 and parameters: {'dropoutRate': 0.3389864899017352, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 66 with value: 0.14356519263755116.
[I 2023-02-17 07:37:32,656] Trial 71 finished with value: 0.14545118618601313 and parameters: {'dropoutRate': 0.2519409066668717, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 66 with value: 0.14356519263755116.
[I 2023-02-17 07:37:39,751] Trial 72 finished with value: 0.1455270527706792 and parameters: {'dropoutRate': 0.24444370570125273, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 66 with value: 0.14356519263755116.
[I 2023-02-17 07:37:46,259] Trial 73 finished with value: 0.15104707974630097 and parameters: {'dropoutRate': 0.31012772712062797, 'hidden_layers': 8, 'laye

[I 2023-02-17 07:40:53,820] Trial 98 finished with value: 0.14984300538587073 and parameters: {'dropoutRate': 0.21374754155946826, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 66 with value: 0.14356519263755116.
[I 2023-02-17 07:41:01,038] Trial 99 finished with value: 0.15083722344651404 and parameters: {'dropoutRate': 0.22598052238625152, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 66 with value: 0.14356519263755116.
[I 2023-02-17 07:41:01,126] A new study created in memory with name: no-name-9db2b59c-1e0b-4803-ae74-74c77df9a570
[I 2023-02-17 07:41:21,174] Trial 0 finished with value: 0.6294109788205888 and parameters: {'dropoutRate': 0.42472407130841755, 'hidden_layers': 12, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True, 'activation': 'swish'}. Best is trial 0 with value: 0.6294109788205888.
[I 2023-02-17 07:41:28,621] Trial 1 finished wi

[I 2023-02-17 07:45:51,800] Trial 26 finished with value: 0.19878583043626147 and parameters: {'dropoutRate': 0.30268045495396867, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True, 'activation': 'relu'}. Best is trial 19 with value: 0.1679103918787506.
[I 2023-02-17 07:46:04,165] Trial 27 finished with value: 0.17136958500163424 and parameters: {'dropoutRate': 0.5005795721808715, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 19 with value: 0.1679103918787506.
[I 2023-02-17 07:46:12,375] Trial 28 finished with value: 0.18194673416696283 and parameters: {'dropoutRate': 0.4151181610665133, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'relu'}. Best is trial 19 with value: 0.1679103918787506.
[I 2023-02-17 07:46:34,630] Trial 29 finished with value: 0.3474042850236098 and parameters: {'dropoutRate': 0.4326610671949147, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks

[I 2023-02-17 07:51:35,998] Trial 55 finished with value: 0.18118355705195832 and parameters: {'dropoutRate': 0.4277426202080971, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 41 with value: 0.16730450937110516.
[I 2023-02-17 07:51:45,453] Trial 56 finished with value: 0.18823176503388417 and parameters: {'dropoutRate': 0.3567842309567071, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'swish'}. Best is trial 41 with value: 0.16730450937110516.
[I 2023-02-17 07:51:57,039] Trial 57 finished with value: 0.18342974440505108 and parameters: {'dropoutRate': 0.5693584482608385, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'relu'}. Best is trial 41 with value: 0.16730450937110516.
[I 2023-02-17 07:52:05,181] Trial 58 finished with value: 0.18137807208889475 and parameters: {'dropoutRate': 0.5171966705201682, 'hidden_layers': 6, 'layer_depth': 2, 'nb

[I 2023-02-17 07:57:08,690] Trial 84 finished with value: 0.1819205330684781 and parameters: {'dropoutRate': 0.47070045366032415, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 41 with value: 0.16730450937110516.
[I 2023-02-17 07:57:16,745] Trial 85 finished with value: 0.1814309788816091 and parameters: {'dropoutRate': 0.41909122185561587, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 41 with value: 0.16730450937110516.
[I 2023-02-17 07:57:29,394] Trial 86 finished with value: 0.17064264002773497 and parameters: {'dropoutRate': 0.39275794623712706, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'relu'}. Best is trial 41 with value: 0.16730450937110516.
[I 2023-02-17 07:57:39,238] Trial 87 finished with value: 0.2585276427368323 and parameters: {'dropoutRate': 0.5428658324696117, 'hidden_layers': 11, 'layer_depth': 2, 'nb

In [42]:
print(line_score)

{'A_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.4322765450401, 'r2': 0.4092989183970269, 'accuracy': 0.6305220883534136, 'f1': 0.5776863269463809}}, 'T_31_O_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.2670041443966863, 'r2': 0.03666547730149605, 'accuracy': 0.8452722063037249, 'f1': 0.4224993794986349}}}


### Training - Classfication (y_class)

In [43]:
def train_fn(model, dl, criterion, optimizer, scheduler, grad_scaler):
    model.train()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.4f"),
    }
    
    for idx, batch in enumerate(dl):
        output_topic = model(batch["feature"].to(torch.int64))
        loss = criterion(F.softmax(output_topic, dim=-1), batch["label"].to(torch.int64))
        # initialization gradients to zero
        optimizer.zero_grad()
        # backprop
        loss.backward()
        # update learning rate
        optimizer.step()
        metrics["loss"].update(loss.item())
        
        if CFG.debug:
            if idx >= 4: break
    
    # update scheduler
    scheduler.step()

    return metrics

def valid_fn(model, dl, criterion):
    model.eval()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.6f"),
    }   
    for idx, batch in enumerate(dl):
        with torch.no_grad():
            output_topic = model(batch["feature"].to(torch.int64))
            loss = criterion(F.softmax(output_topic, dim=-1), batch["label"].to(torch.int64))
        # update metric value
        metrics["loss"].update(loss.item())
    
        if CFG.debug:
            if idx >= 4: break
            
    return metrics

def do_fold_training(BEST_PARAMS={}):
        
    if len(BEST_PARAMS) == 0:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        BEST_PARAMS["dropoutRate"] = 0.5
        BEST_PARAMS["hidden_layers"] = 32
        BEST_PARAMS["layer_depth"] = 2
        BEST_PARAMS["nblocks"] = 2
    else:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        
    model = CustomModel(BEST_PARAMS)
    train_dl = DataLoader(CustomDataset(torch.from_numpy(train_x).to(torch.float32), torch.from_numpy(train_y).to(torch.float32)), shuffle=True, batch_size=CFG.batch_size, drop_last=True)
    valid_dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.from_numpy(valid_y).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)

    # set optimizer
    optimizer_parameters = get_optimizer_params(
        model,
        eta=CFG.eta,
        weight_decay=CFG.weight_decay
    )
    optimizer = AdamW(optimizer_parameters, lr=CFG.eta, weight_decay=CFG.weight_decay)
    scheduler = get_scheduler(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=CFG.epochs
    )
#     scheduler = None
    grad_scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    criterion = nn.CrossEntropyLoss(reduction="mean")
    best_score = np.inf

    early_stopping_cnt = 0
    for epoch in range(CFG.epochs):
        epoch_start_time = time.time()
        # training
        train_metrics = train_fn(model, train_dl, criterion, optimizer, scheduler, grad_scaler)
        # evaluation
        valid_metrics = valid_fn(model, valid_dl, criterion)

        score = valid_metrics["loss"].avg
#         print("Epoch[{0}/{1}]\n train loss : {2}\n valid loss : {3}\n train accuracy : {4}\n valid accuracy : {5}\n eta : {6}\n Elapsed : {7}\n"
#               .format(
#                   epoch+1, CFG.epochs,
#                   round(train_metrics["loss"].avg, 5), round(valid_metrics["loss"].avg, 5),
#                   round(train_metrics["accuracy"].avg, 5), round(valid_metrics["accuracy"].avg, 5),
#                   round(optimizer.param_groups[-1]['lr'], 5), round(time.time() - epoch_start_time, 3)
#               )
#         )
#         print("Epoch[{0}/{1}]\n train loss : {2}\n valid loss : {3}\n eta : {4}\n Elapsed : {5}\n"
#               .format(
#                   epoch+1, CFG.epochs,
#                   round(train_metrics["loss"].avg, 6), round(valid_metrics["loss"].avg, 6),
#                   round(optimizer.param_groups[-1]['lr'], 5), round(time.time() - epoch_start_time, 3)
#               )
#         )
    
        if score < best_score:
            best_score = score
            return_score_dic = {
                "fold": fold,
                "train_loss": train_metrics["loss"].avg,
                "valid_loss": valid_metrics["loss"].avg,
#                 "train_accuracy": train_metrics["accuracy"].avg,
#                 "valid_accuracy": valid_metrics["accuracy"].avg,
            }
            model_save_dic = {'model': model.state_dict()}
            early_stopping_cnt = 0
        else:
            early_stopping_cnt += 1
        
        if early_stopping_cnt == CFG.early_stopping_rounds:
#             print("INFO : Early Stopped ! (Epoch[{0}/{1}])".format(epoch+1, CFG.epochs))
            break
        
    torch.save(
        model_save_dic,
        f"./model_fold{fold}_best.pth",
    )
    model.load_state_dict(torch.load(f"./model_fold{fold}_best.pth")["model"])
    model.eval()

    return copy.deepcopy(model), return_score_dic

In [44]:
class CustomModel(nn.Module):
    def __init__(self, params):
        super(CustomModel, self).__init__()
        self.params = params
        
        self.embedding = nn.Sequential(
            nn.Embedding(params["nfeatures"], embed_dim),
            nn.Flatten(),
        )
        
        self.model = nn.Sequential(
            *[GatedLinearBlock(
                (params["nfeatures"] * embed_dim) if i == 0 else max(4, int(params["hidden_layers"] / (2**(i-1)))),
                dropoutRate=params["dropoutRate"],
                hidden_layers=max(4, int(params["hidden_layers"] / (2**i))),
                layer_depth=params["layer_depth"],
                batch_norm=params["batch_norm"],
                activation=params["activation"]) for i in range(params["nblocks"])
             ],
        )

        self.regressor = nn.Linear(list(self.model.named_parameters())[-1][1].shape[0], 3)
        
    def forward(self, feature):
        x = self.embedding(feature)
        x = self.model(x)
        return self.regressor(x)

In [45]:
kbins_dim = 128
embed_dim = 32
line_split = [
    # A
    ["A_31"],
    # O, T
    ["T_31", "O_31"],
]
score_dic = {
    "mae": None,
    "r2": None,
    "accuracy": None,
    "f1": None,
}
line_split_f1 = {}
line_score = {"_".join(i): {"train": score_dic.copy(), "valid": score_dic.copy()} for i in line_split}
valid_pred = {"_".join(i): None for i in line_split}
test_pred = {"_".join(i): None for i in line_split}

for line in line_split:
    seed_everything()
    
    # Training
    # === Preprocessing (Train) ===
    full_x = df_full[df_full["product_code"].isin(line)]
    full_x["line"] = lbe.transform(full_x["line"])
    
    full_y = full_x["y_class"].values
    full_y_cls = full_x["y_class"].values
    # Drop columns
    full_x = full_x.drop(["product_id", "y_class", "y_quality", "timestamp", "product_code"], axis=1)
    
    full_x = full_x[full_x.columns[~(full_x.var() == 0).values & ~full_x.isna().all().values]]
    full_x = full_x.fillna(-1.0)
    full_x = full_x.T.drop_duplicates().T
    full_x = full_x.drop_duplicates().reset_index(drop=True)
    
    selected_vars = full_x.columns
    cat_vars = ["line"]
    num_vars = diff(selected_vars, cat_vars)
    categoIdx = findIdx(selected_vars, cat_vars)

    full_x[num_vars] = full_x[num_vars].astype("float32")
    full_x[cat_vars] = full_x[cat_vars].astype("int32")
    
    full_x_embed = full_x.copy()
    kbins = KBinsDiscretizer(kbins_dim, encode="ordinal", strategy="uniform")
    full_x_embed[num_vars] = kbins.fit_transform(full_x[num_vars])
    full_x_embed = full_x_embed.astype("int64").to_numpy()

    scaler = MinMaxScaler()
    full_x = scaler.fit_transform(full_x)
    print(full_x.shape)
    
    # === Preprocessing (Test) ===
    test_x = df_test[df_test["product_code"].isin(line)]
    test_x["line"] = lbe.transform(test_x["line"])
    test_x = test_x[selected_vars]
    test_x = test_x.fillna(-1.0)

    test_x_embed = test_x.copy()
    test_x_embed[num_vars] = kbins.transform(test_x[num_vars])
    test_x_embed = test_x_embed.astype("int64").to_numpy()
    
    test_x[num_vars] = test_x[num_vars].astype("float32")
    test_x[cat_vars] = test_x[cat_vars].astype("int32")
    test_x = scaler.transform(test_x)
#     break
    
    # === Neural Architecture Search with Optuna & Training ===
    line_full_pred = np.zeros((len(full_x), 3))
    line_test_pred = np.zeros((len(test_x), 3))
    kfolds_spliter = StratifiedKFold(3 if CFG.debug else 5, shuffle=True, random_state=42)
    for fold, (train_idx, valid_idx) in enumerate(kfolds_spliter.split(full_x, full_y_cls)):

        train_x = full_x_embed[train_idx]
        train_y = full_y[train_idx]
        valid_x = full_x_embed[valid_idx]
        valid_y = full_y[valid_idx]
#         break
        output_container = {"model": None, "optuna_score": np.inf}
        optuna_direction = "minimize"
        optuna_trials = 3 if CFG.debug else 100
        optuna_earlyStopping = Optuna_EarlyStoppingCallback(max(5, 100), direction=optuna_direction)
        optuna_timout = 6 * 3600
        optuna_study = create_study(direction=optuna_direction, sampler=TPESampler(seed=GLOBAL_SEED))

        optuna_study.optimize(
            lambda trial: optuna_objective_function(
                trial, GLOBAL_SEED,
                train_x, train_y, valid_x, valid_y, output_container=output_container,
            ),
            n_jobs=1, n_trials=optuna_trials, timeout=optuna_timout, callbacks=[optuna_earlyStopping]
        )

        model = output_container["model"]
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.zeros(len(valid_x)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"].to(torch.int64)).detach().cpu().numpy())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_full_pred[valid_idx] = softmax(tmp_pred, axis=-1)
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(test_x_embed).to(torch.float32), torch.zeros(len(test_x_embed)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"].to(torch.int64)).detach().cpu().numpy())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_test_pred[:] += softmax(tmp_pred, axis=-1) / kfolds_spliter.n_splits

    # Evaluation
    y_true = full_y.copy()
    y_pred = line_full_pred.copy()
#     eval_mae = mean_absolute_error(y_true, y_pred)
#     eval_r2 = r2_score(y_true, y_pred)

    # Transform quality to class
    y_true_class = y_true.copy()
    y_pred_class = y_pred.argmax(axis=1)
    eval_acc = accuracy_score(y_true_class, y_pred_class)
    eval_f1 = f1_score(y_true_class, y_pred_class, average="macro")
    
    # Save values
    line_score["_".join(line)]["valid"]["mae"] = eval_mae
    line_score["_".join(line)]["valid"]["r2"] = eval_r2
    line_score["_".join(line)]["valid"]["accuracy"] = eval_acc
    line_score["_".join(line)]["valid"]["f1"] = eval_f1
    valid_pred["_".join(line)] = y_pred
    
    # Inference
    y_pred = line_test_pred.copy()
    
    # Save values
    test_pred["_".join(line)] = y_pred

df_meta["cls_cls"] = [valid_pred, test_pred]

for tmp_model in glob("./model_fold*"):
    os.remove(tmp_model)

(249, 1173)


[I 2023-02-17 08:00:00,387] A new study created in memory with name: no-name-a53bdc13-8dde-4c42-a8fa-d2b249e9bf92
[I 2023-02-17 08:00:17,855] Trial 0 finished with value: 1.0761354657319875 and parameters: {'dropoutRate': 0.42472407130841755, 'hidden_layers': 12, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True, 'activation': 'swish'}. Best is trial 0 with value: 1.0761354657319875.
[I 2023-02-17 08:00:25,715] Trial 1 finished with value: 0.9973024084017827 and parameters: {'dropoutRate': 0.5606690070459254, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': True, 'activation': 'relu'}. Best is trial 1 with value: 0.9973024084017827.
[I 2023-02-17 08:00:34,676] Trial 2 finished with value: 0.8412550779489371 and parameters: {'dropoutRate': 0.38254534577572263, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'swish'}. Best is trial 2 with value: 0.8412550779489371.
[I 2023-02-17 08:00:42,181] Trial 3 finished with value: 0.9268280

[I 2023-02-17 08:04:17,759] Trial 28 finished with value: 0.8559185037246118 and parameters: {'dropoutRate': 0.42244792510960183, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'swish'}. Best is trial 24 with value: 0.823057486460759.
[I 2023-02-17 08:04:33,789] Trial 29 finished with value: 1.0588849370296185 and parameters: {'dropoutRate': 0.3144116003149854, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True, 'activation': 'swish'}. Best is trial 24 with value: 0.823057486460759.
[I 2023-02-17 08:04:44,092] Trial 30 finished with value: 0.835547671868251 and parameters: {'dropoutRate': 0.43470416956280367, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'swish'}. Best is trial 24 with value: 0.823057486460759.
[I 2023-02-17 08:04:54,222] Trial 31 finished with value: 0.8361549514990586 and parameters: {'dropoutRate': 0.4182100180020752, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 

[I 2023-02-17 08:10:05,524] Trial 57 finished with value: 0.843722050006573 and parameters: {'dropoutRate': 0.4890389023730089, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'swish'}. Best is trial 52 with value: 0.8075509850795453.
[I 2023-02-17 08:10:16,641] Trial 58 finished with value: 0.8173078573667086 and parameters: {'dropoutRate': 0.6524745224967552, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False, 'activation': 'swish'}. Best is trial 52 with value: 0.8075509850795453.
[I 2023-02-17 08:10:31,566] Trial 59 finished with value: 1.0851993560791016 and parameters: {'dropoutRate': 0.34272206457419535, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': True, 'activation': 'swish'}. Best is trial 52 with value: 0.8075509850795453.
[I 2023-02-17 08:10:43,389] Trial 60 finished with value: 0.990429190488962 and parameters: {'dropoutRate': 0.5252863134760112, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks':

[I 2023-02-17 08:16:05,385] Trial 86 finished with value: 0.8241757750511169 and parameters: {'dropoutRate': 0.28699017196798693, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'swish'}. Best is trial 69 with value: 0.7572109378301181.
[I 2023-02-17 08:16:17,660] Trial 87 finished with value: 0.7719345551270705 and parameters: {'dropoutRate': 0.2434877635578539, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'swish'}. Best is trial 69 with value: 0.7572109378301181.
[I 2023-02-17 08:16:29,976] Trial 88 finished with value: 0.7994335018671476 and parameters: {'dropoutRate': 0.24257015585326086, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False, 'activation': 'swish'}. Best is trial 69 with value: 0.7572109378301181.
[I 2023-02-17 08:16:48,561] Trial 89 finished with value: 1.08111284329341 and parameters: {'dropoutRate': 0.32365404911663054, 'hidden_layers': 10, 'layer_depth': 2, 'nblo

...

[I 2023-02-17 14:20:51,422] Trial 74 finished with value: 0.7044714159435697 and parameters: {'dropoutRate': 0.57796184067918, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'relu'}. Best is trial 41 with value: 0.6891264650556777.
[I 2023-02-17 14:20:58,978] Trial 75 finished with value: 0.7169916099972196 and parameters: {'dropoutRate': 0.6116914828465966, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'swish'}. Best is trial 41 with value: 0.6891264650556777.
[I 2023-02-17 14:21:16,849] Trial 76 finished with value: 0.7371436754862467 and parameters: {'dropoutRate': 0.47686019981410316, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': True, 'activation': 'relu'}. Best is trial 41 with value: 0.6891264650556777.
[I 2023-02-17 14:21:25,222] Trial 77 finished with value: 0.7177384595076243 and parameters: {'dropoutRate': 0.6744383189822485, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 5

[I 2023-02-17 14:25:51,063] Trial 3 finished with value: 0.7597799069351621 and parameters: {'dropoutRate': 0.4736419905302216, 'hidden_layers': 11, 'layer_depth': 2, 'nblocks': 3, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 3 with value: 0.7597799069351621.
[I 2023-02-17 14:26:18,783] Trial 4 finished with value: 0.8101668953895569 and parameters: {'dropoutRate': 0.23903095579116773, 'hidden_layers': 12, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True, 'activation': 'leakyrelu'}. Best is trial 3 with value: 0.7597799069351621.
[I 2023-02-17 14:26:27,869] Trial 5 finished with value: 0.8662444651126862 and parameters: {'dropoutRate': 0.27322294090686733, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': True, 'activation': 'relu'}. Best is trial 3 with value: 0.7597799069351621.
[I 2023-02-17 14:26:49,638] Trial 6 finished with value: 0.7597799599170685 and parameters: {'dropoutRate': 0.5280261676059679, 'hidden_layers': 5, 'layer_depth': 2, 'nbloc

[I 2023-02-17 14:32:22,245] Trial 32 finished with value: 0.7597896489832137 and parameters: {'dropoutRate': 0.49200828550710174, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 4, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 22 with value: 0.7254899375968509.
[I 2023-02-17 14:32:33,891] Trial 33 finished with value: 0.7342891924911075 and parameters: {'dropoutRate': 0.37899459160344545, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 22 with value: 0.7254899375968509.
[I 2023-02-17 14:32:47,402] Trial 34 finished with value: 0.7597841686672635 and parameters: {'dropoutRate': 0.38429335773129675, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 22 with value: 0.7254899375968509.
[I 2023-02-17 14:33:02,411] Trial 35 finished with value: 0.7797023786438836 and parameters: {'dropoutRate': 0.5563490882559552, 'hidden_layers': 10, 'layer_dep

[I 2023-02-17 14:39:44,687] Trial 61 finished with value: 0.7483107381396823 and parameters: {'dropoutRate': 0.4979880394453775, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'relu'}. Best is trial 22 with value: 0.7254899375968509.
[I 2023-02-17 14:39:59,017] Trial 62 finished with value: 0.7597896589173211 and parameters: {'dropoutRate': 0.47108963002023496, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'relu'}. Best is trial 22 with value: 0.7254899375968509.
[I 2023-02-17 14:40:10,531] Trial 63 finished with value: 0.7430389258596632 and parameters: {'dropoutRate': 0.40093831906985616, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'relu'}. Best is trial 22 with value: 0.7254899375968509.
[I 2023-02-17 14:40:24,124] Trial 64 finished with value: 0.7597798870669471 and parameters: {'dropoutRate': 0.31765625578804785, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks

[I 2023-02-17 14:45:57,220] Trial 90 finished with value: 0.7597910298241509 and parameters: {'dropoutRate': 0.40985462980653437, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 4, 'batch_norm': False, 'activation': 'swish'}. Best is trial 74 with value: 0.7203434838189019.
[I 2023-02-17 14:46:10,232] Trial 91 finished with value: 0.7501950131522285 and parameters: {'dropoutRate': 0.4563801268903042, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 74 with value: 0.7203434838189019.
[I 2023-02-17 14:46:24,135] Trial 92 finished with value: 0.7485096686416202 and parameters: {'dropoutRate': 0.4360827674052761, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False, 'activation': 'leakyrelu'}. Best is trial 74 with value: 0.7203434838189019.
[I 2023-02-17 14:46:36,132] Trial 93 finished with value: 0.7492858370145162 and parameters: {'dropoutRate': 0.3656413495521783, 'hidden_layers': 7, 'layer_depth': 2,

In [46]:
print(line_score)

{'A_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.2670041443966863, 'r2': 0.03666547730149605, 'accuracy': 0.714859437751004, 'f1': 0.693691389599318}}, 'T_31_O_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.2670041443966863, 'r2': 0.03666547730149605, 'accuracy': 0.836676217765043, 'f1': 0.36792820252307173}}}


## Meta Learning (no class weight,  normalizing)

In [47]:
df_meta_backup = df_meta.copy()

In [48]:
output_prob_valid = {}
output_prob_test = {}

for line in line_split:
    full_x = []
    test_x = []
    for k, v in df_meta.items():
        full_x.append(v[0]["_".join(line)])
        test_x.append(v[1]["_".join(line)])
    full_x = np.concatenate([i.reshape(-1, 1) if len(i.shape) == 1 else i for i in full_x], axis=1)
    full_x[: ,0] = (full_x[: ,0] - 0.5) * 10
    full_x[: ,4] = (full_x[: ,4]) / 3
    full_y = df_full.loc[df_full["product_code"].isin(line), "y_class"].values
    test_x = np.concatenate([i.reshape(-1, 1) if len(i.shape) == 1 else i for i in test_x], axis=1)
    test_x[: ,0] = (test_x[: ,0] - 0.5) * 10
    test_x[: ,4] = (test_x[: ,4]) / 3

    meta_learner = lm.LogisticRegression(multi_class="multinomial", penalty='elasticnet', solver="saga", l1_ratio=0.5, class_weight=None, random_state=42)
    meta_learner.fit(full_x, full_y)
    output_prob_valid["_".join(line)] = meta_learner.predict_proba(full_x)
    output_prob_test["_".join(line)] = meta_learner.predict_proba(test_x)
    
df_meta["meta_learning"] = [output_prob_valid, output_prob_test]

## Threshold Optimization

### Create infernced value table for analysis

In [49]:
raw_output_container_train = []
raw_output_container_test = []

for line in line_split:
    train_x = []
    test_x = []
    for k, v in df_meta.items():
        train_x.append(v[0]["_".join(line)])
        test_x.append(v[1]["_".join(line)])
    train_x = np.concatenate([i.reshape(-1, 1) if len(i.shape) == 1 else i for i in train_x], axis=1)
    train_y = df_full.loc[df_full["product_code"].isin(line), "y_class"].values
    test_x = np.concatenate([i.reshape(-1, 1) if len(i.shape) == 1 else i for i in test_x], axis=1)
    
    raw_output = pd.DataFrame(train_x, columns=["reg_qual", "cls_cls0", "cls_cls1", "cls_cls2", "reg_cls", "meta_learning_cls0", "meta_learning_cls1", "meta_learning_cls2"])
    raw_output["y_true"] = train_y
    raw_output["product_code"] = df_full.loc[df_full["product_code"].isin(line), "product_code"].values
    raw_output_container_train.append(raw_output)

    raw_output = pd.DataFrame(test_x, columns=["reg_qual", "cls_cls0", "cls_cls1", "cls_cls2", "reg_cls", "meta_learning_cls0", "meta_learning_cls1", "meta_learning_cls2"])
    raw_output["product_code"] = df_test.loc[df_test["product_code"].isin(line), "product_code"].values
    raw_output["product_id"] = df_test.loc[df_test["product_code"].isin(line), "product_id"].values
    raw_output_container_test.append(raw_output)

In [50]:
raw_output_container_train = pd.concat(raw_output_container_train)
raw_output_container_train.head()

,reg_qual,cls_cls0,cls_cls1,cls_cls2,reg_cls,meta_learning_cls0,meta_learning_cls1,meta_learning_cls2,y_true,product_code
0,0.522079,9.980783e-01,0.001922,1.376439e-08,0.217058,0.845271,0.124659,0.030070,0,A_31
1,0.530861,1.137116e-04,0.805275,1.946109e-01,1.024855,0.138040,0.607866,0.254094,1,A_31
2,0.534064,3.202161e-04,0.999670,9.798163e-06,1.058095,0.141185,0.707756,0.151059,2,A_31
3,0.534968,1.227228e-10,0.999998,2.280749e-06,0.753378,0.152543,0.716174,0.131283,1,A_31
4,0.537164,1.573884e-03,0.993443,4.983462e-03,1.196921,0.136001,0.697787,0.166212,1,A_31


In [51]:
with pd.ExcelWriter(architecture_path + "train_threshold_analysis.xlsx") as writer:
    raw_output_container_train.groupby("y_true").describe().T.reset_index().to_excel(writer, sheet_name="all")
    for line in line_split:
        df_tmp = raw_output_container_train[raw_output_container_train["product_code"].isin(line)].groupby("y_true").describe().T.reset_index()
        df_tmp.to_excel(writer, sheet_name="_".join(line))

In [52]:
raw_output_container_test = pd.concat(raw_output_container_test)
raw_output_container_test.head()

,reg_qual,cls_cls0,cls_cls1,cls_cls2,reg_cls,meta_learning_cls0,meta_learning_cls1,meta_learning_cls2,product_code,product_id
0,0.525696,0.336481,0.663172,0.000347,0.390743,0.376795,0.529128,0.094077,A_31,TEST_003
1,0.529598,0.031926,0.967086,0.000988,0.881550,0.161919,0.701884,0.136197,A_31,TEST_004
2,0.530780,0.002096,0.957353,0.040551,1.181436,0.137363,0.685337,0.177300,A_31,TEST_005
3,0.526727,0.219210,0.779565,0.001224,0.604129,0.279060,0.607821,0.113118,A_31,TEST_006
4,0.527295,0.874377,0.093845,0.031778,0.324376,0.778060,0.174290,0.047650,A_31,TEST_007


In [53]:
df_tmp = df_test[["product_id", "product_code"]]
for line in line_split:
    df_tmp.loc[df_tmp["product_code"].isin(line).values, raw_output_container_test.columns[:-1]] = raw_output_container_test[raw_output_container_test["product_code"].isin(line).values].iloc[:, :-1].values

## Inference with Threshold

In [108]:
# # 주혁님 threshold 계산 algorithm
# submit = pd.read_csv('/content/모델4개2.csv', encoding = 'cp949')
# submit['0.748 결과'][(submit['0.748 결과'] != 0) & (submit['class0'] > 0.45) & (submit['Class를 회귀로(모델3개)'] <= 0.75) & (submit['Class를 회귀로(모델1개)'] <= 0.75)] = 0
# s = pd.read_csv('/content/drive/MyDrive/LG_Aimers2/open (7)/sample_submission.csv')
# s['Y_Class'] = submit['0.748 결과']
# s.to_csv('SotaToCha_0.csv',index=False)

In [109]:
def get_threshold_params(x, norm_params={"reg_qual": 0.05, "reg_cls": 0.5, "cls_cls": 0.05, "meta_learning": 0.05}):
    '''
        reg_qual : percentage of reduction on upper and lower bound for class 1
        cls_reg : multiplier of standard deviation on upper and lower bound for class 1 (mean + std * alpha, mean - std * alpha)
        cls_cls : minimum probability for class 1
        meta_learning : minimum probability for class 1 (same as cls_cls)
    '''

    threshold_dic = {"reg_qual": {}, "reg_cls": {}, "cls_cls": {}, "meta_learning": {}}

    df_tmp = x.copy()
    df_tmp["tmp"] = df_tmp["product_code"].apply(lambda x: "A_31" if x == "A_31" else "T_31_O_31")

    # reg_qual
    df_stats = df_tmp.groupby(["tmp", "y_class"])["y_quality"].describe().T
    for k in ["A_31", "T_31_O_31"]:
        tmp_range = (df_stats[k].loc["max", 1] - df_stats[k].loc["min", 1])
        threshold_dic["reg_qual"][k] = [df_stats[k].loc["min", 1] + tmp_range * norm_params["reg_qual"], df_stats[k].loc["max", 1] - tmp_range * norm_params["reg_qual"]]

    # reg_cls
    df_stats = df_tmp.groupby("tmp")["y_class"].describe().T
    for k in ["A_31", "T_31_O_31"]:
        threshold_dic["reg_cls"][k] = [df_stats.loc["mean", k] - df_stats.loc["std", k] * norm_params["reg_cls"], df_stats.loc["mean", k] + df_stats.loc["std", k] * norm_params["reg_cls"]]

    # cls_cls
    df_stats = df_tmp.groupby("tmp")["y_class"].value_counts(normalize=True)
    for k in ["A_31", "T_31_O_31"]:
        threshold_dic["cls_cls"][k] = [df_stats[(k, 1)] * (1 + norm_params["cls_cls"]), 1]

    # meta_learning
    df_stats = df_tmp.groupby("tmp")["y_class"].value_counts(normalize=True)
    for k in ["A_31", "T_31_O_31"]:
        threshold_dic["meta_learning"][k] = [df_stats[(k, 1)] * (1 + norm_params["meta_learning"]), 1]

    for k, v in threshold_dic.items():
        print(k)
        print(v)

    return threshold_dic

In [110]:
def get_inference_label(x, class_spliter):
    df_tmp = x.copy()
    df_tmp["tmp"] = df_tmp["product_code"].apply(lambda x: "A_31" if x == "A_31" else "T_31_O_31")

    # reg_qual
    df_tmp["label_reg_qual"] = 1
    for k, v in class_spliter["reg_qual"].items():
        tmp_labeld = []
        for i in df_tmp.loc[df_tmp["tmp"] == k, "reg_qual"].values:
            if i < v[0]:
                tmp_labeld.append(0)
            elif i > v[1]:
                tmp_labeld.append(2)
            else:
                tmp_labeld.append(1)
        df_tmp.loc[df_tmp["tmp"] == k, "label_reg_qual"] = tmp_labeld

    # reg_cls
    df_tmp["label_reg_cls"] = 1
    for k, v in class_spliter["reg_cls"].items():
        tmp_labeld = []
        for i in df_tmp.loc[df_tmp["tmp"] == k, "reg_cls"].values:
            if i < v[0]:
                tmp_labeld.append(0)
            elif i > v[1]:
                tmp_labeld.append(2)
            else:
                tmp_labeld.append(1)
        df_tmp.loc[df_tmp["tmp"] == k, "label_reg_cls"] = tmp_labeld

    # cls_cls
    df_tmp["label_cls_cls"] = 1
    for k, v in class_spliter["cls_cls"].items():
        tmp_labeld = []
        for i in df_tmp.loc[df_tmp["tmp"] == k, ["cls_cls0", "cls_cls1", "cls_cls2"]].values:
            if (i[1] >= v[0]) & (i[1] <= v[1]):
                tmp_labeld.append(1)
            else:
                tmp_labeld.append(0 if np.argmax([i[0], i[2]]) == 0 else 2)
        df_tmp.loc[df_tmp["tmp"] == k, "label_cls_cls"] = tmp_labeld

    # meta_learning
    df_tmp["label_meta_learning"] = 1
    for k, v in class_spliter["meta_learning"].items():
        tmp_labeld = []
        for i in df_tmp.loc[df_tmp["tmp"] == k, ["meta_learning_cls0", "meta_learning_cls1", "meta_learning_cls2"]].values:
            if (i[1] >= v[0]) & (i[1] <= v[1]):
                tmp_labeld.append(1)
            else:
                tmp_labeld.append(0 if np.argmax([i[0], i[2]]) == 0 else 2)
        df_tmp.loc[df_tmp["tmp"] == k, "label_meta_learning"] = tmp_labeld
    
    # ensemble prediction
    df_tmp["majority_vote"] = df_tmp.filter(regex="label_*").apply(lambda x: Counter(x).most_common(1)[0][0], axis=1)
    
    return df_tmp

In [111]:
params = {"reg_qual": 0.15, "reg_cls": 0.6, "cls_cls": 0.2, "meta_learning": 0.0}
class_spliter = get_threshold_params(df_full, params)

reg_qual
{'A_31': [0.52654928545, 0.53337928555], 'T_31_O_31': [0.5266562694999999, 0.5333929365]}
reg_cls
{'A_31': [0.6089113991488744, 1.4794420145057443], 'T_31_O_31': [0.762420077744715, 1.2605025583584368]}
cls_cls
{'A_31': [0.5686746987951807, 1], 'T_31_O_31': [0.9936962750716332, 1]}
meta_learning
{'A_31': [0.4738955823293173, 1], 'T_31_O_31': [0.828080229226361, 1]}


In [112]:
train_inference = get_inference_label(raw_output_container_train, class_spliter)
for i in ["label_reg_qual", "label_reg_cls", "label_cls_cls", "label_meta_learning", "majority_vote"]:
    print(f"=== {i} ===")
    print("value counts")
    display(train_inference[i].value_counts())
    display(train_inference[i].value_counts(normalize=True))
    display(train_inference.groupby("product_code")[i].value_counts(normalize=True))
    print("\n")
#     print("acc:", accuracy_score(train_inference["y_true"], test_inference[i]))
#     print("f1:", f1_score(train_inference["y_true"], train_inference[i], average="macro"))

=== label_reg_qual ===
value counts


1    390
2    150
0     58
Name: label_reg_qual, dtype: int64

1    0.652174
2    0.250836
0    0.096990
Name: label_reg_qual, dtype: float64

product_code  label_reg_qual
A_31          1                 0.493976
              2                 0.369478
              0                 0.136546
O_31          1                 1.000000
T_31          1                 0.760933
              2                 0.169096
              0                 0.069971
Name: label_reg_qual, dtype: float64



=== label_reg_cls ===
value counts


...

product_code  label_meta_learning
A_31          1                      0.594378
              2                      0.224900
              0                      0.180723
O_31          1                      0.833333
              2                      0.166667
T_31          1                      0.950437
              0                      0.040816
              2                      0.008746
Name: label_meta_learning, dtype: float64



=== majority_vote ===
value counts


1    473
2     66
0     59
Name: majority_vote, dtype: int64

1    0.790970
2    0.110368
0    0.098662
Name: majority_vote, dtype: float64

product_code  majority_vote
A_31          1                0.622490
              2                0.216867
              0                0.160643
O_31          1                1.000000
T_31          1                0.909621
              0                0.055394
              2                0.034985
Name: majority_vote, dtype: float64

In [113]:
for i in ["label_reg_qual", "label_reg_cls", "label_cls_cls", "majority_vote"]:
    print(f"=== {i} ===")
    print("acc:", accuracy_score(train_inference["y_true"], train_inference[i]))
    print("f1:", f1_score(train_inference["y_true"], train_inference[i], average="macro"))

=== label_reg_qual ===
acc: 0.6739130434782609
f1: 0.5694942403968136
=== label_reg_cls ===
acc: 0.7107023411371237
f1: 0.581039119247534
=== label_cls_cls ===
acc: 0.7709030100334449
f1: 0.6527056312934932
=== majority_vote ===
acc: 0.7809364548494984
f1: 0.664250615074791


In [114]:
test_inference = get_inference_label(raw_output_container_test, class_spliter)
for i in ["label_reg_qual", "label_reg_cls", "label_cls_cls", "label_meta_learning", "majority_vote"]:
    print(f"=== {i} ===")
    print("value counts")
    display(test_inference[i].value_counts())
    display(test_inference[i].value_counts(normalize=True))
    display(test_inference.groupby("product_code")[i].value_counts(normalize=True))
    print("\n")
#     print("acc:", accuracy_score(train_inference["y_true"], test_inference[i]))
#     print("f1:", f1_score(train_inference["y_true"], train_inference[i], average="macro"))

=== label_reg_qual ===
value counts


1    271
2     24
0     15
Name: label_reg_qual, dtype: int64

1    0.874194
2    0.077419
0    0.048387
Name: label_reg_qual, dtype: float64

product_code  label_reg_qual
A_31          1                 0.537313
              2                 0.328358
              0                 0.134328
O_31          1                 1.000000
T_31          1                 0.966527
              0                 0.025105
              2                 0.008368
Name: label_reg_qual, dtype: float64



=== label_reg_cls ===
value counts


...

product_code  label_meta_learning
A_31          0                      0.582090
              1                      0.388060
              2                      0.029851
O_31          1                      0.750000
              2                      0.250000
T_31          1                      0.958159
              2                      0.029289
              0                      0.012552
Name: label_meta_learning, dtype: float64



=== majority_vote ===
value counts


1    263
0     44
2      3
Name: majority_vote, dtype: int64

1    0.848387
0    0.141935
2    0.009677
Name: majority_vote, dtype: float64

product_code  majority_vote
A_31          0                0.552239
              1                0.432836
              2                0.014925
O_31          1                1.000000
T_31          1                0.962343
              0                0.029289
              2                0.008368
Name: majority_vote, dtype: float64

In [115]:
test_inference.sort_values("product_id").to_csv(architecture_path + "test_rawoutput.csv", index=False)

## Submission

In [116]:
inference_col = "majority_vote"

In [117]:
df_test_tmp = df_test[["product_id", "line"]]
df_test_tmp["target_class"] = 1

for line in line_split:
    df_test_tmp.loc[df_tmp["product_code"].isin(line), "target_class"] = test_inference.loc[test_inference["product_code"].isin(line), inference_col].values

df_test_tmp["target_class"] = df_test_tmp["target_class"].astype("int32")

In [118]:
df_test_tmp

,product_id,line,target_class
0,TEST_000,T100306,1
1,TEST_001,T100304,1
2,TEST_002,T100304,1
3,TEST_003,T010305,0
4,TEST_004,T010306,1
...,...,...,...
305,TEST_305,T100306,1
306,TEST_306,T100304,1
307,TEST_307,T100306,1
308,TEST_308,T100306,1


In [119]:
submission = pd.read_csv("../datasets/sample_submission.csv")
submission["Y_Class"] = df_test_tmp["target_class"].values

In [120]:
submission

,PRODUCT_ID,Y_Class
0,TEST_000,1
1,TEST_001,1
2,TEST_002,1
3,TEST_003,0
4,TEST_004,1
...,...,...
305,TEST_305,1
306,TEST_306,1
307,TEST_307,1
308,TEST_308,1


In [121]:
submission.to_csv(architecture_path + "submission_" + architecture_name + ".csv", index=False)